# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import datetime
import warnings
warnings.filterwarnings("ignore")
import abc
from sys import getsizeof
from skimage.transform import resize
from imageio import imread

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!unzip -q /content/gdrive/MyDrive/Gesture_Recognition/Project_data.zip

In [5]:
!ls

gdrive	Project_data  sample_data


In [6]:
!ls "Project_data"

train  train.csv  val  val.csv


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [7]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [8]:

x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

In [11]:
#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 8)  32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 8)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 60, 60, 16)   6

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model_a.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 4.9167 - categorical_accuracy: 0.2760Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-11-1514_14_47.366825/model-00001-4.91670-0.27602-1.45638-0.37000.h5
67/67 [==============================] - 419s 6s/step - loss: 4.9167 - categorical_accuracy: 0.2760 - val_loss: 1.4564 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.2877 - categorical_accuracy: 0.2886
Epoch 00002: saving model to model_init_2021-11-1514_14_47.366825/model-00002-2.28771-0.28856-1.44919-0.42000.h5
67/67 [==============================] - 146s 2s/step - loss: 2.2877 - categorical_accuracy: 0.2886 - val_loss: 1.4492 - val_categorical_accuracy: 0.4200 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.9661 - categorical_accuracy: 0.2985
Epoch 00003: saving 

In [16]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [ ]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 120, 120, 32)  2624      
                                                                 
 activation_8 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_9 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 10, 40, 40, 32)   0         
 3D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 10, 40, 40, 32)    0         
                                                      

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.8721 - categorical_accuracy: 0.2036Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-11-1514_14_47.366825/model-00001-1.87213-0.20362-1.60878-0.23000.h5
67/67 [==============================] - 417s 6s/step - loss: 1.8721 - categorical_accuracy: 0.2036 - val_loss: 1.6088 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6104 - categorical_accuracy: 0.1990
Epoch 00002: saving model to model_init_2021-11-1514_14_47.366825/model-00002-1.61040-0.19900-1.60868-0.24000.h5
67/67 [==============================] - 157s 2s/step - loss: 1.6104 - categorical_accuracy: 0.1990 - val_loss: 1.6087 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6107 - categorical_accuracy: 0.1990
Epoch 00003: saving 

### Changing x,y,z values Experiment [1]

In [ ]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [ ]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 60, 60, 32)    2624      
                                                                 
 activation (Activation)     (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_1 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 10, 20, 20, 32)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 10, 20, 20, 32)    0         
                                                        

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6036 - categorical_accuracy: 0.2353Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-11-1514_14_47.366825/model-00001-1.60361-0.23529-1.54463-0.33000.h5
67/67 [==============================] - 382s 6s/step - loss: 1.6036 - categorical_accuracy: 0.2353 - val_loss: 1.5446 - val_categorical_accuracy: 0.3300 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5196 - categorical_accuracy: 0.3582
Epoch 00002: saving model to model_init_2021-11-1514_14_47.366825/model-00002-1.51955-0.35821-1.55259-0.23000.h5
67/67 [==============================] - 191s 3s/step - loss: 1.5196 - categorical_accuracy: 0.3582 - val_loss: 1.5526 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.5539 - categorical_accuracy: 0.2438
Epoch 00003: saving 

### Changing Batch size to 20 Experiment [2]

In [ ]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.1034 - categorical_accuracy: 0.5157Source path =  Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2021-11-1514_14_47.366825/model-00001-1.10341-0.51575-1.82532-0.37000.h5
67/67 [==============================] - 455s 7s/step - loss: 1.1034 - categorical_accuracy: 0.5157 - val_loss: 1.8253 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.2471 - categorical_accuracy: 0.5075
Epoch 00002: saving model to model_init_2021-11-1514_14_47.366825/model-00002-1.24713-0.50746-1.25879-0.44500.h5
67/67 [==============================] - 204s 3s/step - loss: 1.2471 - categorical_accuracy: 0.5075 - val_loss: 1.2588 - val_categorical_accuracy: 0.4450 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.1882 - categorical_accuracy: 0.5075
Epoch 00003: saving 

### Changing Batch size to 40 Experiment [3]

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.4654 - categorical_accuracy: 0.8342 Source path =  Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-11-1514_14_47.366825/model-00001-0.46541-0.83420-1.30332-0.47083.h5
67/67 [==============================] - 892s 13s/step - loss: 0.4654 - categorical_accuracy: 0.8342 - val_loss: 1.3033 - val_categorical_accuracy: 0.4708 - lr: 7.8125e-06
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.4706 - categorical_accuracy: 0.8335
Epoch 00002: saving model to model_init_2021-11-1514_14_47.366825/model-00002-0.47065-0.83348-1.24041-0.50000.h5
67/67 [==============================] - 630s 10s/step - loss: 0.4706 - categorical_accuracy: 0.8335 - val_loss: 1.2404 - val_categorical_accuracy: 0.5000 - lr: 7.8125e-06
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.4580 - categorical_accura

### Change Optimizer to Adadelta Experiment [4]

In [ ]:
model_b.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_12 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_13 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_13 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 10, 20, 20, 32)   0         
 g3D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.4390 - categorical_accuracy: 0.8400 Source path =  Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-11-1514_14_47.366825/model-00001-0.43898-0.84000-1.36536-0.46250.h5
67/67 [==============================] - 881s 13s/step - loss: 0.4390 - categorical_accuracy: 0.8400 - val_loss: 1.3654 - val_categorical_accuracy: 0.4625 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.4491 - categorical_accuracy: 0.8421
Epoch 00002: saving model to model_init_2021-11-1514_14_47.366825/model-00002-0.44906-0.84206-1.28738-0.48000.h5
67/67 [==============================] - 638s 10s/step - loss: 0.4491 - categorical_accuracy: 0.8421 - val_loss: 1.2874 - val_categorical_accuracy: 0.4800 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.4512 - categorical_accuracy: 0.84

### Change epoch to 20 Experiment [5]

In [ ]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [ ]:
model_b.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 60, 60, 32)    2624      
                                                                 
 activation (Activation)     (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_1 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 10, 20, 20, 32)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 10, 20, 20, 32)    0         
                                                        

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.6195 - categorical_accuracy: 0.2099Source path =  Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-11-1805_58_48.010909/model-00001-1.61954-0.20986-1.60223-0.17083.h5
67/67 [==============================] - 770s 11s/step - loss: 1.6195 - categorical_accuracy: 0.2099 - val_loss: 1.6022 - val_categorical_accuracy: 0.1708 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.6166 - categorical_accuracy: 0.2009
Epoch 00002: saving model to model_init_2021-11-1805_58_48.010909/model-00002-1.61656-0.20086-1.59843-0.22500.h5
67/67 [==============================] - 532s 8s/step - loss: 1.6166 - categorical_accuracy: 0.2009 - val_loss: 1.5984 - val_categorical_accuracy: 0.2250 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.6089 - categorical_accuracy: 0.2169

### Model C Experiment [6]

In [18]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_c = Sequential()

model_c.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_c.add(Flatten())

model_c.add(Dense(nb_dense[0], activation='relu'))
model_c.add(Dropout(0.5))

model_c.add(Dense(nb_dense[1], activation='relu'))
model_c.add(Dropout(0.5))

#softmax layer
model_c.add(Dense(nb_dense[2], activation='softmax'))
model_c.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_4 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 15, 60, 60, 8)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_5 (Batc  (None, 15, 60, 60, 16)  

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.5998 - categorical_accuracy: 0.1810Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-11-1805_58_48.010909/model-00001-2.59980-0.18100-1.61906-0.21000.h5
67/67 [==============================] - 278s 4s/step - loss: 2.5998 - categorical_accuracy: 0.1810 - val_loss: 1.6191 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.6152 - categorical_accuracy: 0.1940
Epoch 00002: saving model to model_init_2021-11-1805_58_48.010909/model-00002-2.61522-0.19403-1.63286-0.22000.h5
67/67 [==============================] - 113s 2s/step - loss: 2.6152 - categorical_accuracy: 0.1940 - val_loss: 1.6329 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.4683 - categorical_accuracy: 0.2338
Epoch 00003: saving 

### Model D Experiment [7]

In [19]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 120, 120, 8)   224       
                                                                 
 batch_normalization_8 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_8 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 15, 60, 60, 8)    0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_9 (Batc  (None, 15, 60, 60, 16)  

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 3.0607 - categorical_accuracy: 0.2097Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-11-1805_58_48.010909/model-00001-3.06074-0.20965-1.62144-0.19000.h5
67/67 [==============================] - 257s 4s/step - loss: 3.0607 - categorical_accuracy: 0.2097 - val_loss: 1.6214 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 2.8839 - categorical_accuracy: 0.2289
Epoch 00002: saving model to model_init_2021-11-1805_58_48.010909/model-00002-2.88393-0.22886-1.63507-0.20000.h5
67/67 [==============================] - 108s 2s/step - loss: 2.8839 - categorical_accuracy: 0.2289 - val_loss: 1.6351 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 2.8090 - categorical_accuracy: 0.2139
Epoch 00003: saving 

## Model E Experiment [8]

In [20]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_e = Sequential()

model_e.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

model_e.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

model_e.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

model_e.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_e.add(Flatten())

model_e.add(Dense(nb_dense[0], activation='relu'))
model_e.add(Dropout(0.5))

model_e.add(Dense(nb_dense[1], activation='relu'))
model_e.add(Dropout(0.5))

#softmax layer
model_e.add(Dense(nb_dense[2], activation='softmax'))
model_e.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_12 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_12 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_13 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_13 (Bat  (None, 15, 60, 60, 16)  

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.7705 - categorical_accuracy: 0.2036Source path =  Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-11-1805_58_48.010909/model-00001-2.77047-0.20362-1.60192-0.24000.h5
67/67 [==============================] - 267s 4s/step - loss: 2.7705 - categorical_accuracy: 0.2036 - val_loss: 1.6019 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.6048 - categorical_accuracy: 0.2289
Epoch 00002: saving model to model_init_2021-11-1805_58_48.010909/model-00002-2.60475-0.22886-1.61474-0.25000.h5
67/67 [==============================] - 113s 2s/step - loss: 2.6048 - categorical_accuracy: 0.2289 - val_loss: 1.6147 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.7246 - categorical_accuracy: 0.2239
Epoch 00003: saving 

### Model F Experiment [9]

In [21]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_16 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_16 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_16 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_16 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_17 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_17 (Bat  (None, 15, 60, 60, 16)  

In [22]:
batch_size = 20
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 3.2452 - categorical_accuracy: 0.2100Source path =  Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2021-11-1817_43_46.183495/model-00001-3.24517-0.20997-1.64296-0.18000.h5
67/67 [==============================] - 380s 5s/step - loss: 3.2452 - categorical_accuracy: 0.2100 - val_loss: 1.6430 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 2.9851 - categorical_accuracy: 0.2040
Epoch 00002: saving model to model_init_2021-11-1817_43_46.183495/model-00002-2.98513-0.20398-1.66914-0.16000.h5
67/67 [==============================] - 162s 2s/step - loss: 2.9851 - categorical_accuracy: 0.2040 - val_loss: 1.6691 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 3.1185 - categorical_accuracy: 0.2239
Epoch 00003: saving 

### Model G Experiment [10] 

In [23]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(Activation('relu'))
model_g.add(Dropout(0.25))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_20 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_20 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_20 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_20 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_21 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_21 (Bat  (None, 15, 60, 60, 16)  

In [24]:
batch_size = 30
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 3.6002 - categorical_accuracy: 0.2113Source path =  Project_data/val ; batch size = 30
Batch:  4 Index: 30

Epoch 00001: saving model to model_init_2021-11-1817_43_46.183495/model-00001-3.60019-0.21132-1.62513-0.25625.h5
67/67 [==============================] - 356s 5s/step - loss: 3.6002 - categorical_accuracy: 0.2113 - val_loss: 1.6251 - val_categorical_accuracy: 0.2562 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 3.2722 - categorical_accuracy: 0.2388
Epoch 00002: saving model to model_init_2021-11-1817_43_46.183495/model-00002-3.27217-0.23881-1.66529-0.18000.h5
67/67 [==============================] - 113s 2s/step - loss: 3.2722 - categorical_accuracy: 0.2388 - val_loss: 1.6653 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.3960 - categorical_accuracy: 0.1841


### Final Model

In [ ]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

batch_size = 40
num_epochs = 20

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = 'adam' #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_7 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_8 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 15, 60, 60, 8)    0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_8 (Batc  (None, 15, 60, 60, 16)  

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.8099 - categorical_accuracy: 0.3148Source path =  Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-11-1811_52_10.520291/model-00001-1.80992-0.31478-1.58957-0.18750.h5
67/67 [==============================] - 700s 10s/step - loss: 1.8099 - categorical_accuracy: 0.3148 - val_loss: 1.5896 - val_categorical_accuracy: 0.1875 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.2618 - categorical_accuracy: 0.4489
Epoch 00002: saving model to model_init_2021-11-1811_52_10.520291/model-00002-1.26178-0.44893-1.56113-0.34000.h5
67/67 [==============================] - 476s 7s/step - loss: 1.2618 - categorical_accuracy: 0.4489 - val_loss: 1.5611 - val_categorical_accuracy: 0.3400 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.1186 - categorical_accuracy: 0.5277